In [1]:
# Import necessary packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [2]:
def grayscale(img):
    # Convert mpimg color space into 1-channel grayscale
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [3]:
def gaussian_blur(img, kernel_size=5):
    # Apply Gaussian blur with default kernel size
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [4]:
def canny(img, low_threshold, high_threshold):
    # Canny edge detection
    return cv2.Canny(img, low_threshold, high_threshold)

In [5]:
def polygon_roi(img):
    if len(img.shape) == 3:
        r, c, _ = img.shape
        color_mask = (255, 255, 255)
    else:
        r, c = img.shape
        color_mask = 255
    offset_lb = c // 10
    offset_rb = c // 20
    offset_lt = c // 20
    offset_rt = c // 20
    offset_bottom = 0
    offset_top = r // 11
    vertices = np.array([[
        (0 + offset_lb, r + offset_bottom),           # bottom left
        (c // 2 - offset_lt, r // 2 + offset_top),    # top left
        (c // 2 + offset_rt, r // 2 + offset_top),    # top right
        (c - offset_rb, r + offset_bottom),           # bottom right
    ]], dtype=np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, color_mask)
    masked = cv2.bitwise_and(img, mask)
    return masked

In [6]:
def hough_lines(img):
    # Distance resolution of the Hough grid (in pixels)
    rho = 2
    # Angular resolution of the Hough grid (in radians)
    theta = np.pi / 180
    # Min. # of votes required for a line
    threshold = 5
    # Min. # of pixels required for a line
    min_line_len = 20
    # Maximum gap between connectable line segments
    max_line_gap = 40
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

In [7]:
def detect_lines(img):
    # Obtain grayscale image
    gray = grayscale(img)
    # Apply Gaussian blur filter
    blurred = gaussian_blur(gray, 3)
    # Detect Canny edges
    edges = canny(blurred, 50, 200)
    # Mask region of interest
    roi = polygon_roi(edges)
    # Detect Hough lines
    lines = hough_lines(roi)
    return lines

In [8]:
def select_lines(lines, min_angle, max_angle, min_x, max_x):
    # Calculate tangent for each angle
    min_gradient = np.tan(np.radians(min_angle))
    max_gradient = np.tan(np.radians(max_angle))
    selected = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            # First we check the position of the line
            if min(x1, x2) < min_x or max(x1, x2) > max_x:
                continue
            # Vertical lines are not likely to be lane lines
            if x1 == x2:
                continue
            # We subtract y2 from y1 since y-axis is upside down
            m = (y1 - y2) / (x2 - x1)
            # For those selected lines, keep the following:
            # (1) gradient, (2) y-intercept, (3) segnemtn length,
            if min_gradient <= m <= max_gradient:
                selected.append(line)
    return selected

In [9]:
def aggregate_lines(lines):
    gradients, intercepts, weights = [], [], []
    min_y, max_y = 0, 560
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            # Keep min & max y values
            min_y = min(y1, y2) if min_y == 0 or min(y1, y2) < min_y else min_y
            max_y = max(y1, y2) if max(y1, y2) > max_y else max_y
            m = (y2 - y1) / (x2 - x1)
            b = y1 - m * x1
            # Use each line's length as its weight in averaging
            l = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            gradients.append(m)
            intercepts.append(b)
            weights.append(l)
    # Now we compute weighted (based on line length) average of the
    # gradient and y-intercept
    g, i, w = np.array(gradients), np.array(intercepts), np.array(weights)
    avg_m = np.average(g, weights=w)
    avg_b = np.average(i, weights=w)
    # Calculate low & high endpoints
    x1 = int((max_y - avg_b) / avg_m)
    x2 = int((min_y - avg_b) / avg_m)
    return x1, max_y, x2, min_y

In [10]:
def draw_lines(img, lines, c=(255, 0, 0), t=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), c, t)
    return img

In [11]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [12]:
def process_image(img):
    r = img.shape[0]
    c = img.shape[1]
    lines = detect_lines(img)
    # print("# of found lines: ", len(lines))
    left_candidates = select_lines(lines, 30, 45, 0, c // 2)
    # print("# of left lane line candidates: ", len(left_candidates))
    right_candidates = select_lines(lines, -45, -30, c // 2, c)
    # print("# of right lane line candidates: ", len(right_candidates))

    line_img = np.zeros_like(img)
    # line_img = draw_lines(line_img, left_candidates, c=(0, 255, 0))
    # line_img = draw_lines(line_img, right_candidates, c=(255, 255, 0))
    if len(left_candidates) > 0:
        left_lane_line = aggregate_lines(left_candidates)
        line_img = draw_lines(line_img, [(left_lane_line, )], c=(255, 0, 0), t=10)
    if len(right_candidates) > 0:
        right_lane_line = aggregate_lines(right_candidates)
        line_img = draw_lines(line_img, [(right_lane_line, )], c=(0, 0, 255), t=10)
    annotated = weighted_img(line_img, img)
    return annotated

In [13]:
import os

filenames = os.listdir("test_images")
for filename in filenames:
    pathname = os.path.join(os.getcwd(), "test_images", filename)
    # Read image file
    image = mpimg.imread(pathname)
    lane_line_detection = process_image(image)
    cv2_image = cv2.cvtColor(lane_line_detection, cv2.COLOR_RGB2BGR)
    output_pathname = os.path.join(os.getcwd(), "test_images_output", filename)
    cv2.imwrite(output_pathname, cv2_image)

In [14]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

filenames = os.listdir("test_videos")
for filename in filenames:
    pathname = os.path.join(os.getcwd(), "test_videos", filename)
    output_pathname = os.path.join(os.getcwd(), "test_videos_output", filename)
    clip = VideoFileClip(pathname)
    output_clip = clip.fl_image(process_image)
    %time output_clip.write_videofile(output_pathname, audio=False)

Imageio: 'ffmpeg.linux64' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64 (27.2 MB)


Downloading: 8192/28549024 bytes (0.024576/28549024 bytes (0.1%90112/28549024 bytes (0.3%139264/28549024 bytes (0.5188416/28549024 bytes (0.7245760/28549024 bytes (0.9303104/28549024 bytes (1.1360448/28549024 bytes (1.3376832/28549024 bytes (1.3450560/28549024 bytes (1.6499712/28549024 bytes (1.8548864/28549024 bytes (1.9598016/28549024 bytes (2.1647168/28549024 bytes (2.3696320/28549024 bytes (2.4761856/28549024 bytes (2.7827392/28549024 bytes (2.9892928/28549024 bytes (3.1958464/28549024 bytes (3.41024000/28549024 bytes (3.6%1089536/28549024 bytes (3.8%1155072/28549024 bytes (4.0%1187840/28549024 bytes (4.2%1269760/28549024 bytes (4.4%1327104/28549024 bytes (4.6%1368064/28549024 bytes (4.8%1417216/28549024 bytes (5.0%1466368/28549024 bytes (5.1%1499136/28549024 bytes (5.3%1564672/28549024 bytes (5.5%1597440/28549024 bytes (5.6%1630208/28549024 bytes (5.7%1679360/28549024 bytes (5.9%1728512/28549024 bytes (6.1%1777664/28549024 bytes (6.2%1826816/28549024 bytes (6.4%1875968/28549024 by

12099584/28549024 bytes (42.4%12132352/28549024 bytes (42.5%12181504/28549024 bytes (42.7%12230656/28549024 bytes (42.8%12279808/28549024 bytes (43.0%12328960/28549024 bytes (43.2%12378112/28549024 bytes (43.4%12410880/28549024 bytes (43.5%12460032/28549024 bytes (43.6%12509184/28549024 bytes (43.8%12558336/28549024 bytes (44.0%12607488/28549024 bytes (44.2%12656640/28549024 bytes (44.3%12705792/28549024 bytes (44.5%12754944/28549024 bytes (44.7%12787712/28549024 bytes (44.8%12836864/28549024 bytes (45.0%12886016/28549024 bytes (45.1%12935168/28549024 bytes (45.3%12984320/28549024 bytes (45.5%13033472/28549024 bytes (45.7%13082624/28549024 bytes (45.8%13131776/28549024 bytes (46.0%13180928/28549024 bytes (46.2%13230080/28549024 bytes (46.3%13295616/28549024 bytes (46.6%13361152/28549024 bytes (46.8%13426688/28549024 bytes (47.0%13475840/28549024 bytes (47.2%13557760/28549024 bytes (47.5%13606912/28549024 bytes (47.7%13656064/28549024 bytes (47.8%13721600/28549024 bytes (48.1%13770752/

24764416/28549024 bytes (86.7%24797184/28549024 bytes (86.9%24829952/28549024 bytes (87.0%24846336/28549024 bytes (87.0%24879104/28549024 bytes (87.1%24911872/28549024 bytes (87.3%24944640/28549024 bytes (87.4%24977408/28549024 bytes (87.5%25010176/28549024 bytes (87.6%25042944/28549024 bytes (87.7%25075712/28549024 bytes (87.8%25108480/28549024 bytes (87.9%25141248/28549024 bytes (88.1%25190400/28549024 bytes (88.2%25239552/28549024 bytes (88.4%25272320/28549024 bytes (88.5%25337856/28549024 bytes (88.8%25387008/28549024 bytes (88.9%25436160/28549024 bytes (89.1%25485312/28549024 bytes (89.3%25534464/28549024 bytes (89.4%25583616/28549024 bytes (89.6%25632768/28549024 bytes (89.8%25681920/28549024 bytes (90.0%25739264/28549024 bytes (90.2%25796608/28549024 bytes (90.4%25845760/28549024 bytes (90.5%25927680/28549024 bytes (90.8%25976832/28549024 bytes (91.0%26009600/28549024 bytes (91.1%26058752/28549024 bytes (91.3%26091520/28549024 bytes (91.4%26140672/28549024 bytes (91.6%26189824/

100%|█████████▉| 221/222 [00:06<00:00, 36.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.28 s, sys: 300 ms, total: 2.58 s
Wall time: 7.03 s
[MoviePy] >>>> Building video /src/test_videos_output/challenge.mp4
[MoviePy] Writing video /src/test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:15<00:00, 15.24it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/challenge.mp4 

CPU times: user 5.46 s, sys: 554 ms, total: 6.01 s
Wall time: 17 s
[MoviePy] >>>> Building video /src/test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video /src/test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:21<00:00, 31.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/solidYellowLeft.mp4 

CPU times: user 7.76 s, sys: 891 ms, total: 8.65 s
Wall time: 22.5 s
